In [15]:
using JuAFEM, Tensors, TimerOutputs, ProgressMeter
using KrylovMethods, IterativeSolvers, SparseArrays

In [8]:
struct NeoHooke
    μ::Float64
    λ::Float64
end

function Ψ(C, mp::NeoHooke)
    μ = mp.μ
    λ = mp.λ
    Ic = tr(C)
    J = sqrt(det(C))
    return μ / 2 * (Ic - 3) - μ * log(J) + λ / 2 * log(J)^2
end

function constitutive_driver(C, mp::NeoHooke) 
    # to compute all derivatives in one function call 
    ∂²Ψ∂C², ∂Ψ∂C = Tensors.hessian(y -> Ψ(y, mp), C, :all)
    S = 2.0 * ∂Ψ∂C
    ∂S∂C = 4.0 * ∂²Ψ∂C²
    return S, ∂S∂C
end

constitutive_driver (generic function with 1 method)

In [9]:
struct ScratchValues{T, CV <: CellValues, FV <: FaceValues, TT <: AbstractTensor, dim, Ti}
    Ke::Matrix{T}
    ge::Vector{T}
    cellvalues::CV
    facevalues::FV
    global_dofs::Vector{Int}
    assembler::JuAFEM.AssemblerSparsityPattern{T, Ti}
end;

In [10]:
function create_values(refshape, dim, order::Int)
    # Interpolations and values
    interpolation_space = Lagrange{dim, refshape, 1}()
    quadrature_rule = QuadratureRule{dim, refshape}(order)
    face_quadrature_rule = QuadratureRule{dim-1, refshape}(order)
    cellvalues = [CellVectorValues(quadrature_rule, interpolation_space) for i in 1:Threads.nthreads()];
    facevalues = [FaceVectorValues(face_quadrature_rule, interpolation_space) for i in 1:Threads.nthreads()];
    return cellvalues, facevalues
end;

In [11]:
function create_scratchvalues(K, f, dh::DofHandler{dim}) where {dim}
    nthreads = Threads.nthreads()
    assemblers = [start_assemble(K, f) for i in 1:nthreads] #preparing all the threads for the assembly
    cellvalues, facevalues = create_values(RefTetrahedron, dim, 1) #by this function we create CellValues to all the threads

    n_basefuncs = getnbasefunctions(cellvalues[1])
    global_dofs = [zeros(Int, ndofs_per_cell(dh)) for i in 1:nthreads] #defining the dofs vector for all the threads

    fes = [zeros(n_basefuncs) for i in 1:nthreads] # Local force vector
    Kes = [zeros(n_basefuncs, n_basefuncs) for i in 1:nthreads] #defining the local stiffness matirx for all the threads

    return [ScratchValues(Kes[i], fes[i], cellvalues[i], facevalues[i], global_dofs[i],
                          assemblers[i]) for i in 1:nthreads]
end; 

In [12]:
function assemble_cell!(scratch::ScratchValues, cell::Int, dh::DofHandler, ue)
    ke,ge,cellvalues,facevalues,global_dofs,assembler = scratch.ke, scratch.ge,scratch.cellvalues,
                                                        scratch.facevalues,scratch.global_dofs,scratch.assembler
    
    fill!(ke,0.0)
    fill!(ge,0.0)
    
    b = Vec{3}((0.0, -0.5, 0.0))
    t = Vec{3}((0.1,0.0,0.0))
    ndofs = getnbasefunctions(cellvalues)
    
        for qp in 1:getnquadpoints(cellvalues)
        dΩ = getdetJdV(cellvalues, qp)
        # Compute deformation gradient F and right Cauchy-Green tensor C
        ∇u = function_gradient(cellvalues, qp, ue)
        F = one(∇u) + ∇u
        C = tdot(F)
        # Compute stress and tangent
        S, ∂S∂C = constitutive_driver(C, mp)
        P = F ⋅ S
        I = one(S)
        ∂P∂F = otimesu(F, I) ⊡ ∂S∂C ⊡ otimesu(F', I) + otimesu(I, S)

        # Loop over test functions
        for i in 1:ndofs
            # Test function + gradient
            δui = shape_value(cellvalues, qp, i)
            ∇δui = shape_gradient(cellvalues, qp, i)
            # Add contribution to the residual from this test function
            ge[i] += ( ∇δui ⊡ P - δui ⋅ b ) * dΩ

            ∇δui∂P∂F = ∇δui ⊡ ∂P∂F # Hoisted computation
            for j in 1:ndofs
                ∇δuj = shape_gradient(cellvalues, qp, j)
                # Add contribution to the tangent
                ke[i, j] += ( ∇δui∂P∂F ⊡ ∇δuj ) * dΩ
            end
        end
    end

    # Surface integral for the traction
    for face in 1:nfaces(cell)
        if onboundary(cell, face)
            reinit!(facevalues, cell, face)
            for q_point in 1:getnquadpoints(facevalues)
                dΓ = getdetJdV(facevalues, q_point)
                for i in 1:ndofs
                    δui = shape_value(facevalues, q_point, i)
                    ge[i] -= (δui ⋅ t) * dΓ
                end
            end
        end
    end
    
    celldofs!(global_dofs, dh, cell)
    assemble!(assembler, global_dofs, ge, Ke)

end

assemble_cell! (generic function with 1 method)

In [16]:
function doassemble(K::SparseMatrixCSC, colors, grid::Grid, dh::DofHandler, ue) where {dim}

    g = zeros(ndofs(dh))
    scratches = create_scratchvalues(K, f, dh)

    for color in colors
        # Each color is safe to assemble threaded
        Threads.@threads for i in 1:length(color)
            assemble_cell!(scratches[Threads.threadid()], color[i], dh, ue)
        end
    end

    return K, f
end

doassemble (generic function with 1 method)

In [20]:
function create_colored_grid(celltype, n)
    grid = generate_grid(celltype, (n, n, n), Vec{3}((0.0, 0.0, 0.0)), Vec{3}((1.0, 1.0, 1.0)))
    cell_colors, final_colors = JuAFEM.create_coloring(grid)
    return grid, final_colors
end

create_colored_grid (generic function with 1 method)

In [17]:
function solve()
    reset_timer!()
    
    #generate a grid
    N = 10
    L = 1.0 
    left = zero(Vec{3})
    right = L * ones(Vec{3})
    grid, colors = create_colored_grid(Tetrahedron,N)
    
    #Material parameters 
    E = 10.0
    v = 0.3 
    μ = E / (2(1 + v))
    λ = (E * v) / ((1 + v) * (1 - 2v))
    mp = NeoHooke(μ, λ)
    
    #finite element base 
    ip = Lagrange{3, RefTetrahedron, 1}() #lagrange shape functions of dimension 3 and 1st oder 
    qr = QuadratureRule{3, RefTetrahedron}(1) #The gauss points of 3D element of order 1 
    qr_face = QuadratureRule{2, RefTetrahedron}(1) #the gauss points of the faces which is 2D and of order 1 
    cv = CellVectorValues(qr, ip) #we combine the shape functions and the gauss points to get an object 
    fv = FaceVectorValues(qr_face, ip)
    
    #dof handler 
    dh = DofHandler(grid)
    push!(dh, :u, 3)
    close!(dh)
    
    function rotation(X, t, θ = deg2rad(60.0))
        x, y, z = X
        return t * Vec{3}(
            (0.0,
            L/2 - y + (y-L/2)*cos(θ) - (z-L/2)*sin(θ),
            L/2 - z + (y-L/2)*sin(θ) + (z-L/2)*cos(θ)
            ))
    end
    
    dbcs = ConstraintHandler(dh)
    dbc = Dirichlet(:u, getfaceset(grid, "right"), (x,t) -> [0.0, 0.0, 0.0], [1, 2, 3])
    add!(dbcs, dbc)
    dbc = Dirichlet(:u, getfaceset(grid, "left"), (x,t) -> rotation(x,t) , [1, 2, 3])
    add!(dbcs, dbc)
    close!(dbcs)
    t = 0.5
    JuAFEM.update!(dbcs, t)
    
    # pre allocotion of vectors for the solution of Newton increments 
    _ndofs = ndofs(dh)
    un = zeros(_ndofs)
    u = zeros(_ndofs)
    Δu = zeros(_ndofs)
    ΔΔu = zeros(_ndofs)
    apply!(un, dbcs)
    
    #create sparse matrix and residual vector 
    K = create_sparsity_pattern(dh)
    g = zeros(_ndofs)
    
    #perform Newton Iterations 
    newton_itr = -1
    NEWTON_TOL = 1e-8
    prog = ProgressMeter.ProgressThresh(NEWTON_TOL, "solving:")
    
    while true; newton_itr += 1
        u .= un .+ Δu #current guess 
        doassemble(K, colors, grid, u)
        normg = norm(g[JuAFEM.free_dofs(dbcs)])
        apply_zero!(K, g, dbcs)
        ProgressMeter.update!(prog, normg; showvalues = [(:iter, newton_itr)])

        if normg < NEWTON_TOL
            break
        elseif newton_itr > 30
            error("Reached maximum Newton iterations, aborting")
        end

        # Compute increment using cg! from IterativeSolvers.jl
        @timeit "linear solve (KrylovMethods.cg)" ΔΔu′, flag, relres, iter, resvec = KrylovMethods.cg(K, g; maxIter = 1000)
        @assert flag == 0
        @timeit "linear solve (IterativeSolvers.cg!)" IterativeSolvers.cg!(ΔΔu, K, g; maxiter=1000)

        apply_zero!(ΔΔu, dbcs)
        Δu .-= ΔΔu
    end

    # Save the solution
    @timeit "export" begin
        vtk_grid("hyperelasticity", dh) do vtkfile
            vtk_point_data(vtkfile, dh, u)
        end
    end

    print_timer(title = "Analysis with $(getncells(grid)) elements", linechars = :ascii)
    return u
end

solve (generic function with 1 method)

In [18]:
u = solve()

UndefVarError: UndefVarError: colors not defined